<a href="https://colab.research.google.com/github/Hutzii/Deeplabv3/blob/main/DeepLabv3_self.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#데이터셋 다운로드 사람이미지를 이용

In [2]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [15]:
!gdown https://drive.google.com/uc?id=1B9A9UCJYMwTL4oBEo4RZfbMZMaZhKJaz
!unzip -q instance-level-human-parsing.zip

Downloading...
From: https://drive.google.com/uc?id=1B9A9UCJYMwTL4oBEo4RZfbMZMaZhKJaz
To: /content/instance-level-human-parsing.zip
100% 2.91G/2.91G [00:27<00:00, 107MB/s]


In [16]:
# from PIL import Image
# im = Image.open('./instance-level_human_parsing/instance-level_human_parsing/Training/Images/0005986.jpg')
# width, height = im.size
# print(width,height)
#!ls ./instance-level_human_parsing/instance-level_human_parsing/Training/Images
!file ./instance-level_human_parsing/instance-level_human_parsing/Training/Category_ids/0012111.png

./instance-level_human_parsing/instance-level_human_parsing/Training/Category_ids/0012111.png: PNG image data, 333 x 500, 8-bit grayscale, non-interlaced


In [17]:
IMAGE_SIZE = 512
BATCH_SIZE = 4
NUM_CLASSES = 20
DATA_DIR = './instance-level_human_parsing/instance-level_human_parsing/Training'
NUM_TRAIN_IMAGES = 1000
NUM_VAL_IMAGES = 50


In [18]:
train_images = sorted(glob(os.path.join(DATA_DIR,'Images/*')))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(os.path.join(DATA_DIR,'Category_ids/*')))[:NUM_TRAIN_IMAGES]

val_images = sorted(glob(os.path.join(DATA_DIR,'Images/*')))[NUM_TRAIN_IMAGES:NUM_VAL_IMAGES+NUM_TRAIN_IMAGES]
val_masks = sorted(glob(os.path.join(DATA_DIR,'Category_ids/*')))[NUM_TRAIN_IMAGES:NUM_VAL_IMAGES+NUM_TRAIN_IMAGES]



In [19]:
def read_image(image_path, mask=False):
  image = tf.io.read_file(image_path)
  if mask:
    image = tf.image.decode_png(image,channels=1)#mask는 gray로 되어있을것이니 chane이 1이 맞다.
    image.set_shape([None,None,1])
    image = tf.image.resize(images=image, size=[IMAGE_SIZE,IMAGE_SIZE])
  else:
    image = tf.image.decode_png(image,channels=3)#mask는 gray로 되어있을것이니 chane이 1이 맞다.
    image.set_shape([None,None,3])
    image = tf.image.resize(images=image, size=[IMAGE_SIZE,IMAGE_SIZE])
    image = image / 127.5 - 1 # 255를 절반으로 나눈 127.5로 나누고거기에 1을빼준 -1~1 까지의 값이 생긴다.
  return image


In [20]:
def load_data(image_list,mask_list):
  image = read_image(image_list)
  mask = read_image(mask_list,mask=True)
  return image, mask

In [21]:
def data_generator(image_list,mask_list):
  dataset = tf.data.Dataset.from_tensor_slices((image_list,mask_list))
  dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
  return dataset

In [22]:
train_dataset = data_generator(train_images,train_masks)
val_dataset = data_generator(val_images,val_masks)

print("Train" , train_dataset)
print("val ", val_dataset)

Train <_BatchDataset element_spec=(TensorSpec(shape=(4, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 512, 512, 1), dtype=tf.float32, name=None))>
val  <_BatchDataset element_spec=(TensorSpec(shape=(4, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 512, 512, 1), dtype=tf.float32, name=None))>


In [ ]:
##모델구성

In [23]:
def convolution_block(block_input, num_filters=256, kernel_size=3,dilation_rate=1,padding='same',use_bias=False):
  x = layers.Conv2D(num_filters,kernel_size,dilation_rate=dilation_rate,padding='same',
                    use_bias=use_bias,kernel_initializer=keras.initializers.HeNormal())(block_input)
  x = layers.BatchNormalization()(x)
  return tf.nn.relu(x)


In [24]:
def DilatedSpatialPyramidPooling(dspp_input):
  dims = dspp_input.shape
  x = layers.AveragePooling2D(pool_size=(dims[-3],dims[-2]))(dspp_input)
  x = convolution_block(x,kernel_size=1,use_bias=True)

  out_pool = layers.UpSampling2D(size=(dims[-3] // x.shape[1],dims[-2] // x.shape[2]),
                                 interpolation='bilinear')(x)

  out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
  out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
  out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
  out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

  x = layers.Concatenate(axis=-1)([out_pool,out_1,out_6,out_12,out_18])
  output = convolution_block(x, kernel_size=1)
  return output


In [ ]:
#모델구성 Decoder

In [25]:
def DeeplabV3(image_size, num_classes):
  model_input = keras.Input(shape=(image_size,image_size,3))
  resnet50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)

  x = resnet50.get_layer('conv4_block6_2_relu').output
  x = DilatedSpatialPyramidPooling(x)
  input_a = layers.UpSampling2D(size=(image_size//4 // x.shape[1],
                                      image_size//4 // x.shape[2]),
                                interpolation = 'bilinear')(x)

  input_b = resnet50.get_layer('conv2_block3_2_relu').output
  input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

  x = layers.Concatenate(axis=-1)([input_a,input_b])
  x = convolution_block(x)
  x = convolution_block(x)
  x = layers.UpSampling2D(size=(image_size // x.shape[1],
                                image_size // x.shape[2]),
                          interpolation='bilinear')(x)
  model_output = layers.Conv2D(num_classes, kernel_size=(1,1), padding='same')(x)
  return keras.Model(inputs=model_input, outputs=model_output)

model = DeeplabV3(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()


94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [ ]:
##모델 학습

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss= loss,
              metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=val_dataset, epochs=20)

Epoch 1/20
250/250 [==============================] - 1418s 6s/step - loss: 1.2054 - accuracy: 0.6385 - val_loss: 2.7545 - val_accuracy: 0.5960
Epoch 2/20
250/250 [==============================] - 1397s 6s/step - loss: 0.9550 - accuracy: 0.6974 - val_loss: 1.9739 - val_accuracy: 0.5965
Epoch 3/20
 95/250 [==========>...................] - ETA: 14:11 - loss: 0.8769 - accuracy: 0.7254